In [4]:
# train_yolo.py
# This script fine-tunes a pre-trained YOLOv8 model on a custom dataset.

from ultralytics import YOLO
import torch
import os
import yaml
from pathlib import Path # For robust path handling

def train_model():
    """
    Trains a YOLOv8 model on a custom dataset.
    """
    # --- Configuration ---
    # Path to your data.yaml file - should be relative to this script's location
    # or an absolute path. Based on your structure: c:/Users/adity/Downloads/yolo_room_detection/dataset_yolo/data.yaml
    # If this script is in c:/Users/adity/Downloads/yolo_room_detection/, then 'dataset_yolo/data.yaml' is correct.
    data_yaml_path = 'dataset_yolo/data.yaml'

    # Choose a pre-trained model (e.g., yolov8n.pt, yolov8s.pt, yolov8m.pt)
    # 'n' (nano) is fastest, 'x' (extra-large) is most accurate but slowest.
    # 's' (small) or 'm' (medium) are good balances.
    model_name_to_train = 'yolov8s.pt' # Using .pt loads pre-trained weights

    # Training parameters
    epochs = 100  # Number of training epochs. Start smaller (e.g., 25-50) to test setup.
                   # Increase for better performance (e.g., 100-300).
    batch_size = 16 # Batch size. Adjust based on your GPU memory.
                    # Use -1 for YOLO's auto-batch feature if unsure, but can be slow on some systems.
    img_size = 640 # Input image size (pixels). Common sizes are 320, 416, 640.
    project_name = 'yolo_room_detection_project' # Root folder where training runs will be saved
    experiment_name = 'run1' # Subfolder for this specific training run (e.g., yolov8s_custom_run1)
                             # Results will be in 'yolo_room_detection_project/run1/'

    # --- Validate data.yaml path ---
    absolute_data_yaml_path = Path(data_yaml_path).resolve()
    if not absolute_data_yaml_path.exists():
        print(f"Error: Data YAML file not found at '{absolute_data_yaml_path}'")
        print(f"Current working directory: {Path.cwd()}")
        print("Please ensure the path is correct and the file exists.")
        print("This path should point to your dataset configuration file (e.g., dataset_yolo/data.yaml).")
        return

    # --- Load and Validate data.yaml content ---
    try:
        with open(absolute_data_yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)
        print(f"\n=== Dataset Configuration ===")
        print(f"Successfully loaded data configuration from: {absolute_data_yaml_path}")

        # Validate essential keys in data_config
        required_keys = ['train', 'val', 'names'] # 'path' is also common and recommended
        if 'path' not in data_config:
            print("Warning: 'path' key not found in data.yaml. Assuming paths in 'train' and 'val' are relative to the data.yaml file's parent directory or absolute.")
            # If 'path' is missing, paths in 'train', 'val', 'test' are usually relative to the 'data.yaml' parent directory.
            # Ultralytics handles this, but it's good practice to have 'path'.
            # For example:
            # path: ../dataset_yolo  # Or absolute path to dataset_yolo
            # train: train/images
            # val: valid/images
            # test: test/images
            # Or (if data.yaml is inside dataset_yolo):
            # path: . # Current directory
            # train: train/images

        for key in required_keys:
            if key not in data_config:
                print(f"Error: Missing required key '{key}' in {absolute_data_yaml_path.name}")
                return

        # Ultralytics handles path resolution relative to data.yaml, but we can print for verification
        dataset_root_from_yaml = Path(data_config.get('path', '.')).resolve() # Default to current dir if 'path' not in yaml
        if not Path(data_config.get('path', '.')).is_absolute() : # If 'path' in yaml is relative
             dataset_root_from_yaml = (absolute_data_yaml_path.parent / data_config.get('path', '.')).resolve()

        print(f"Dataset root path (derived from data.yaml 'path' or data.yaml location): {dataset_root_from_yaml}")
        # Ultralytics will resolve train/val paths relative to the YAML file's location and its 'path' field
        print(f"Expected training images directory (relative to YAML & 'path'): {data_config['train']}")
        print(f"Expected validation images directory (relative to YAML & 'path'): {data_config['val']}")
        print(f"Number of classes defined: {len(data_config.get('names', []))}")
        print(f"Class names: {data_config.get('names', [])}")

        if not data_config.get('names'):
            print("Warning: 'names' field in data.yaml is empty. The model will train, but class names won't be resolved correctly during evaluation or later use without the YAML.")

    except Exception as e:
        print(f"Error reading or parsing data.yaml at '{absolute_data_yaml_path}': {e}")
        return

    # --- Initialize YOLO Model ---
    try:
        model = YOLO(model_name_to_train)
        print(f"\n=== Model Initialization ===")
        print(f"Successfully loaded/initialized model: {model_name_to_train}")
    except Exception as e:
        print(f"Error loading/initializing YOLO model '{model_name_to_train}': {e}")
        print("Ensure the model name is correct (e.g., 'yolov8s.pt' for pre-trained, 'yolov8s.yaml' for from scratch).")
        print("If using a .pt file, it will be downloaded if not found locally in a standard cache location.")
        return

    # --- Check for GPU ---
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    if device == 'cpu':
        print("Warning: Training on CPU will be very slow. Consider using a GPU for faster training.")
        # Optionally adjust batch size for CPU if it's too high
        if batch_size > 8 and batch_size != -1 : # -1 means auto-batch
            print(f"Current batch size is {batch_size}. Consider reducing it for CPU training (e.g., 4 or 8).")


    # --- Start Training ---
    print(f"\n--- Starting YOLOv8 Training ---")
    print(f"Dataset YAML: {absolute_data_yaml_path}")
    print(f"Model: {model_name_to_train}")
    print(f"Epochs: {epochs}")
    print(f"Batch size: {batch_size if batch_size != -1 else 'Auto'}")
    print(f"Image size: {img_size}x{img_size}")
    print(f"Project directory: ./{project_name}") # Saved in current working dir under project_name
    print(f"Experiment name: {experiment_name}")
    print(f"Device: {device}")
    print("---------------------------------")

    try:
        # The train() method handles the training loop, data loading, augmentation, etc.
        results = model.train(
            data=str(absolute_data_yaml_path), # Path to the data.yaml file
            epochs=epochs,          # Total number of training epochs
            batch=batch_size,       # Batch size for training (-1 for auto batch)
            imgsz=img_size,         # Input image size for training
            project=project_name,   # Name of the project directory to save results
            name=experiment_name,   # Name of the experiment run (sub-directory under project)
            device=device,          # Device to run on (e.g., 'cpu', '0' for GPU 0, '0,1' for multiple GPUs)
            exist_ok=False,         # False: Error if experiment_name already exists. True: Overwrite.
            # patience=20,          # Early stopping patience: epochs to wait for no improvement before stopping
            # lr0=0.01,             # Initial learning rate
            # lrf=0.01,             # Final learning rate (lr0 * lr0)
            # optimizer='AdamW',    # Optimizer: 'SGD', 'Adam', 'AdamW', 'NAdam', 'RAdam', 'RMSProp'
            # augment=True,         # Enable/disable default data augmentation
            # workers=8,            # Dataloader workers (adjust based on CPU cores)
            # Other hyperparameters can be set here. Refer to Ultralytics documentation.
        )
        print("\n--- Training Completed ---")
        # The best model is usually saved as 'best.pt' in the experiment's weights directory
        # results.save_dir should be project_name/experiment_name
        best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
        print(f"Results, logs, and checkpoints saved to: {Path(results.save_dir).resolve()}")
        print(f"Best performing model weights saved at: {best_model_path.resolve()}")
        print(f"You can use this '{best_model_path.name}' file (path: {best_model_path.resolve()}) for inference in detect_objects.py.")

    except Exception as e:
        print(f"An error occurred during training: {e}")
        import traceback
        traceback.print_exc()
        print("\nTroubleshooting Tips:")
        print("- Ensure your dataset is correctly formatted (images and YOLO .txt labels).")
        print("- Double-check all paths in your data.yaml file and ensure they are correct relative to the YAML file itself or absolute.")
        print("- Verify the 'names' list in data.yaml matches the class indices in your label files.")
        print("- If using a GPU, check GPU memory (nvidia-smi). Try reducing 'batch_size' or 'imgsz'.")
        print("- Ensure all dependencies are correctly installed ('pip install ultralytics torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cuXXX' for CUDA users).")

if __name__ == '__main__':
    # This ensures the script runs only when executed directly (not imported as a module)

    print("--- YOLOv8 Custom Model Training Script ---")
    print("Prerequisites:")
    print("1. Your custom dataset (images and YOLO format labels) must be prepared in 'dataset_yolo/'.")
    print("2. A 'data.yaml' file ('dataset_yolo/data.yaml') configuring your dataset paths and class names must exist.")
    print("   - Ensure 'path', 'train', 'val', and 'names' are correctly set in 'data.yaml'.")
    print("   - Example 'data.yaml' content (if data.yaml is in 'dataset_yolo/'):")
    print("     path: .  # relative path to the dataset directory from this yaml file")
    print("     train: train/images")
    print("     val: valid/images")
    print("     test: test/images  # Optional")
    print("     names:")
    print("       - class0_name")
    print("       - class1_name")
    print("       # ... and so on for all your classes")
    print("3. All required libraries (torch, ultralytics, etc.) must be installed.")
    print("-------------------------------------------\n")

    # Call the training function
    train_model()

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# train_yolo.py
# This script fine-tunes a pre-trained YOLOv8 model on a custom dataset (CPU version).

from ultralytics import YOLO
import torch
import os
import yaml
from pathlib import Path

def train_model():
    """
    Trains a YOLOv8 model on a custom dataset (CPU optimized version).
    """
    # --- Configuration ---
    data_yaml_path = 'dataset_yolo/data.yaml'  # Keep your dataset path
    model_name_to_train = 'yolov8s.pt'

    # Training parameters (adjusted for CPU)
    epochs = 100
    batch_size = 8  # Reduced from 16 for CPU stability
    img_size = 640
    project_name = 'yolo_room_detection_result'  # Changed output path
    experiment_name = 'cpu_run12'  # New experiment name

    # --- Validate data.yaml path ---
    absolute_data_yaml_path = Path(data_yaml_path).resolve()
    if not absolute_data_yaml_path.exists():
        print(f"Error: Data YAML file not found at '{absolute_data_yaml_path}'")
        print(f"Current working directory: {Path.cwd()}")
        return

    # --- Load and Validate data.yaml content ---
    try:
        with open(absolute_data_yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)
        print(f"\n=== Dataset Configuration ===")
        print(f"Successfully loaded data configuration from: {absolute_data_yaml_path}")

        required_keys = ['train', 'val', 'names']
        for key in required_keys:
            if key not in data_config:
                print(f"Error: Missing required key '{key}' in {absolute_data_yaml_path.name}")
                return

        # Print dataset information
        print(f"Number of classes defined: {len(data_config.get('names', []))}")
        print(f"Class names: {data_config.get('names', [])}")

    except Exception as e:
        print(f"Error reading or parsing data.yaml: {e}")
        return

    # --- Initialize YOLO Model ---
    try:
        model = YOLO(model_name_to_train)
        print(f"\n=== Model Initialization ===")
        print(f"Successfully loaded model: {model_name_to_train}")
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # --- Force CPU usage ---
    device = 'cpu'  # Explicitly set to CPU
    print(f"\n--- Hardware Configuration ---")
    print(f"Using device: {device.upper()}")
    print("Note: Training on CPU may be slower than GPU")

    # --- Start Training ---
    print(f"\n--- Starting YOLOv8 Training ---")
    print(f"Output Directory: {project_name}/{experiment_name}")
    print(f"Batch size: {batch_size} (CPU optimized)")
    print(f"Image size: {img_size}x{img_size}")
    print("---------------------------------")

    try:
        results = model.train(
            data=str(absolute_data_yaml_path),
            epochs=epochs,
            batch=batch_size,
            imgsz=img_size,
            project=project_name,
            name=experiment_name,
            device=device,
            exist_ok=False,
            workers=4  # Reduced workers for CPU stability
        )

        # Training completion
        print("\n--- Training Completed ---")
        best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
        print(f"Results saved to: {Path(results.save_dir).resolve()}")
        print(f"Best model: {best_model_path.resolve()}")

    except Exception as e:
        print(f"Training error: {e}")
        import traceback
        traceback.print_exc()
        print("\nTroubleshooting Tips:")
        print("- Reduce batch_size if you encounter memory issues")
        print("- Verify dataset paths in data.yaml")
        print("- Check available disk space")

if __name__ == '__main__':
    print("--- YOLOv8 CPU Training Script ---")
    print("Modified Configuration:")
    print(f"- Output Path: yolo_room_detection_result/cpu_run12")
    print(f"- Forced CPU Execution")
    print("----------------------------------\n")
    train_model()

In [3]:
!pip  install ultralytics

In [1]:
# yolo_diagnostic_script.py
# This script helps diagnose common issues with YOLOv8, PyTorch, and CUDA setups.

import torch
import os
from pathlib import Path
from ultralytics import YOLO

def run_diagnostics():
    """
    Runs a series of diagnostic checks for the YOLO/PyTorch environment.
    """
    print("--- Starting YOLO Environment Diagnostics ---")

    # --- 1. PyTorch and CUDA Check ---
    print("\n--- 1. PyTorch & CUDA Diagnostics ---")
    try:
        print(f"PyTorch version: {torch.__version__}")
        cuda_available = torch.cuda.is_available()
        print(f"CUDA available: {cuda_available}")

        if cuda_available:
            print(f"CUDA version (linked with PyTorch): {torch.version.cuda}")
            print(f"Number of GPUs available: {torch.cuda.device_count()}")
            current_gpu_id = torch.cuda.current_device()
            print(f"Current GPU ID: {current_gpu_id}")
            gpu_name = torch.cuda.get_device_name(current_gpu_id)
            print(f"GPU Name: {gpu_name}")
            total_mem = torch.cuda.get_device_properties(current_gpu_id).total_memory / (1024**3)
            reserved_mem = torch.cuda.memory_reserved(current_gpu_id) / (1024**3)
            allocated_mem = torch.cuda.memory_allocated(current_gpu_id) / (1024**3)
            free_mem = total_mem - allocated_mem # More intuitive free memory
            print(f"GPU Total Memory: {total_mem:.2f} GB")
            print(f"GPU Reserved Memory: {reserved_mem:.2f} GB")
            print(f"GPU Allocated Memory: {allocated_mem:.2f} GB")
            print(f"GPU Free Memory (approx): {free_mem:.2f} GB")
        else:
            print("CUDA not available. PyTorch is running on CPU.")
            print("If you have an NVIDIA GPU, ensure drivers and CUDA toolkit compatible with PyTorch are installed.")
    except Exception as e:
        print(f"Error during PyTorch/CUDA check: {e}")

    # --- 2. Basic YOLOv8 Model Loading ---
    print("\n--- 2. Basic YOLOv8 Model Loading Test ---")
    try:
        print("Attempting to load a standard YOLOv8n model (yolov8n.pt)...")
        # This will download yolov8n.pt if not already cached
        model_nano = YOLO('yolov8n.pt')
        print("Successfully loaded YOLOv8n (yolov8n.pt).")
        print(f"Model type: {type(model_nano)}")
        # You could add a simple inference test here if needed
        # print("Attempting a dummy inference with yolov8n.pt...")
        # _ = model_nano.predict(torch.zeros(1, 3, 640, 640), verbose=False) # Dummy tensor
        # print("Dummy inference with yolov8n.pt successful.")

    except Exception as e:
        print(f"Error loading standard YOLOv8n model: {e}")
        print("This could indicate an issue with the ultralytics installation or network access if downloading.")

    # --- 3. Custom Model Loading (Update Path if Necessary) ---
    print("\n--- 3. Custom Model Loading Test ---")
    # IMPORTANT: Update this path to your actual custom model file
    custom_model_path_str = 'yolo_room_detection_project/run1/weights/best.pt'
    custom_model_path = Path(custom_model_path_str).resolve()

    print(f"Attempting to load custom model from: {custom_model_path}")
    if custom_model_path.exists():
        try:
            custom_model = YOLO(custom_model_path)
            print(f"Successfully loaded custom model: {custom_model_path.name}")
            print(f"Custom model type: {type(custom_model)}")
            if hasattr(custom_model, 'names'):
                print(f"Custom model class names: {custom_model.names}")
            else:
                print("Custom model does not have 'names' attribute readily available (might be loaded differently or an issue).")
        except Exception as e:
            print(f"Error loading custom model from '{custom_model_path}': {e}")
            print("Possible issues:")
            print("  - The .pt file might be corrupted.")
            print("  - There might be a mismatch in the YOLO version used for training vs. current ultralytics version (less common for .pt files).")
            print("  - Insufficient memory (though usually a more direct OOM error for this).")
    else:
        print(f"Custom model file NOT FOUND at: {custom_model_path}")
        print("Please verify the 'custom_model_path_str' variable in this script matches your trained model's location.")
        print(f"Current working directory is: {Path.cwd()}")


    # --- 4. OpenCV Check (Often used with YOLO) ---
    print("\n--- 4. OpenCV Version Check ---")
    try:
        import cv2
        print(f"OpenCV version: {cv2.__version__}")
        # Test a basic OpenCV function
        # dummy_image = cv2.imread('dataset_yolo/test/images/-24-_jpg.rf.325010ad98a53cc13c237cad8af5b213.jpg') # Replace with an actual small image path if available
        # if dummy_image is not None:
        #    print("OpenCV successfully read a test image.")
        # else:
        #    print("OpenCV could not read a test image (check path or image integrity).")
    except ImportError:
        print("OpenCV (cv2) is not installed. This is usually required for image/video processing with YOLO.")
    except Exception as e:
        print(f"Error during OpenCV check: {e}")


    print("\n--- Diagnostics Complete ---")
    print("Review the output above for any errors or warnings.")
    print("If 'CUDA available' is False and you have an NVIDIA GPU, re-check drivers and CUDA installation.")
    print("If basic YOLOv8n loading fails, there might be an issue with your 'ultralytics' library installation.")
    print("If custom model loading fails, check the model path and integrity.")

if __name__ == '__main__':
    run_diagnostics()


--- Starting YOLO Environment Diagnostics ---

--- 1. PyTorch & CUDA Diagnostics ---
PyTorch version: 2.6.0+cpu
CUDA available: False
CUDA not available. PyTorch is running on CPU.
If you have an NVIDIA GPU, ensure drivers and CUDA toolkit compatible with PyTorch are installed.

--- 2. Basic YOLOv8 Model Loading Test ---
Attempting to load a standard YOLOv8n model (yolov8n.pt)...


100%|██████████| 6.25M/6.25M [00:04<00:00, 1.63MB/s]


Successfully loaded YOLOv8n (yolov8n.pt).
Model type: <class 'ultralytics.models.yolo.model.YOLO'>

--- 3. Custom Model Loading Test ---
Attempting to load custom model from: C:\Users\adity\Downloads\yolo_room_detection\yolo_room_detection_project\run1\weights\best.pt
Custom model file NOT FOUND at: C:\Users\adity\Downloads\yolo_room_detection\yolo_room_detection_project\run1\weights\best.pt
Please verify the 'custom_model_path_str' variable in this script matches your trained model's location.
Current working directory is: c:\Users\adity\Downloads\yolo_room_detection

--- 4. OpenCV Version Check ---
OpenCV version: 4.11.0

--- Diagnostics Complete ---
Review the output above for any errors or warnings.
If 'CUDA available' is False and you have an NVIDIA GPU, re-check drivers and CUDA installation.
If basic YOLOv8n loading fails, there might be an issue with your 'ultralytics' library installation.
If custom model loading fails, check the model path and integrity.
